In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from utils import *
from models import *

In [7]:
################### unload parameters #################
with open('parameters.json') as json_file:
    parameters = json.load(json_file)
    
low_res_name = parameters['low_res_name']
super_res_name = parameters['super_res_name']
att_path = parameters['att_path']
adj_path = parameters['adj_path']
batch_size = parameters['batch_size']
device = "cuda" if torch.cuda.is_available() else "cpu"

################### load data #################
print('Load Datasets...')

att_low_res_path = att_path+low_res_name+'.npy'
adj_low_res_path = adj_path+low_res_name+'.npy'
att_super_res_path = att_path+super_res_name+'.npy'
adj_super_res_path = adj_path+super_res_name+'.npy'
_, _, dataset_test = load_data(att_low_res_path, adj_low_res_path, att_super_res_path, adj_super_res_path)

Load Datasets...


In [40]:
## load data
X_low = torch.from_numpy(np.load(att_low_res_path)).float()
X_super = torch.from_numpy(np.load(att_super_res_path)).float()

In [8]:
model = GraphSR(dataset_test.adj_super.size(0)).to(device)
criterion = nn.L1Loss().to(device)
model.load_state_dict(torch.load(f'model_state/graphSR_{low_res_name}_{super_res_name}'))

<All keys matched successfully>

In [10]:
loss, pred_super, gt_super, gt_low = evaluation(model, criterion, device, batch_size, dataset_test)
print(f'Test Loss: {round(loss, 4)}')

Test Loss: 0.0222


In [43]:
geodata_low_res = pd.read_csv('../data-processing/reality/geodata/nta.csv')
geodata_super_res = pd.read_csv('../data-processing/reality/geodata/taxi.csv')

In [44]:
index = np.random.randint(0,874,1)
gt_low = gt_low.cpu()
gt_super = gt_super.cpu()
pred_super = pred_super.cpu()
geodata_low_res['count_gt'] = gt_low[index][0].detach().numpy()*torch.max(X_low).item()
geodata_super_res['count_gt'] = gt_super[index][0].detach().numpy()*torch.max(X_super).item()
geodata_super_res['count_pred'] = pred_super[index][0].detach().numpy()*torch.max(X_super).item()
geodata_super_res['diff'] = torch.mean((pred_super - gt_super),dim=0).detach().numpy()*torch.max(X_super).item()

In [45]:
geodata_low_res.to_csv(f'visualization/vis_low_{low_res_name}_{super_res_name}.csv', header=True)
geodata_super_res.to_csv(f'visualization/vis_super_{low_res_name}_{super_res_name}.csv', header=True)